# Completing the Game


In the previous lectures I gave a demonstration of how I use diagraming to help think through the classes, objects, and
interactions I want to see in a software system. This is part of a larger area of programming called _Software
Engineering_, and I've just given a short overview of two popular diagrams we use in software engineering: the _Class
Diagram_, which shows our classes along with their attributes and functions, and the _Interaction Diagram_, which shows
how classes are instantiated into objects in the system, and how methods bound to those objects are called.

This of course is just the tip of the iceberg, and there is a lot to think about with diagramming such as managing
asynchronous calls, diagraming network relationships, modeling inheritance relationships, and managing or keeping up to
date all of the different kinds of diagrams that your project starts to collect. I'm not going to go into these issues
here, I just wanted to give you a little taste of how programmers tend to think in diagrams when building out their
software solutions. I personally believe that you should take a pragmatic approach to diagramming -- use it when it
helps you think about the relationships between objects in your software system, and use it as a language to describe
your system requirements to other programmers. But don't feel you need to restrict yourself based on what is in your
initial diagrams, or that your diagrams need to completely encompass your complete project. Your diagrams should be
living documents, and should be updated as you add new features or change interaction flows in your software.

But, for this project I did do up a final set of diagrams and some code, so why don't I walk you through that?


![An image which shows the class and sequence diagrams created in previous
lectures.](1-3-capturing_interactions_between_classes.svg)


In [ ]:
# You've already seen the Adventurer and Item classes, so let's do just a quick review.
# One of the changes I've made here is tweaking a few of the values, for instance the
# hit points of the Adventurer I've decided should range between 10 and 30 plus the
# adventurer constitution level, and I want to start the adventurer off with a small
# stick in their bag.

import random


class Adventurer:
    """Represents the user in our game, a fearless Adventurer.

    The Adventurer moves throughout the Cave system going from Room to Room trying to slay Monsters.

    Attributes:
        name: The display name for the character.
        strength: Strength, on a scale of 1-18.
        constitution: Constitution, on a scale of 1-18.
        hit_points: An integer, when <=0 the character is dead.
        bag: Which holds an unsorted list of Item objects the character has on them.
    """

    # Just like the class definition, each method or function associated with a class
    # should have a triple quoted string that describes it and the arguments it takes.
    # For methods, the first argument is always self, which is a reference to the object
    # being called, and we do not include that in the args list.
    def __init__(self, name: str) -> None:
        """Initializes the Adventurer with a name and some random stats.

        Args:
          name: The name of the Adventurer.
        """
        # Set the name to whatever the caller has passed in. Remember our field and our
        # argument can be the same, as is the case with the "name" here. We distinguish
        # between these by using self.name to refer to the object attribute field.
        self.name: str = name

        # Create default values for constitution and strength between 3-18
        self.strength: int = random.randint(3, 18)
        self.constitution: int = random.randint(3, 18)

        # Let's set the hit points of the character to be the constitution + a random offset
        self.hit_points: int = self.constitution + random.randint(10, 30)

        # The bag starts with just a little stick
        self.bag: list[Item] = [Item("A small stick", 0, 1)]


# I haven't changed the Item class at all, but I want to reiterate how important it is to
# do error checking on the input values, even though I've omitted that here for brevity.


class Item:
    """An item which can be held by an Adventurer.

    Items are weapons in this game, and may have a range of possible damages they can inflict.

    Attributes:
        name: The display name for the item.
        min_dmg: The minimum damage an item can cause, cannot be negative.
        max_dmg: The maximum damage an item can cause, cannot be negative.
    """

    def __init__(self, name: str, min_dmg: int, max_dmg: int) -> None:
        """Initializes the Item and it's minimum and maximum damage.

        Args:
            name: A description of the Item being used.
            min_dmg: The minimum damage the item can cause.
            max_dmg: The maximum damage the item can cause.
        """
        # For the Adventurer class we didn't do any input error checking, but frankly we
        # probably should, so let's do that here for items. If the user passes in an item
        # without a name let's set it to unknown
        if name != None:
            self.name: str = name
        else:
            self.name: str = "Unknown Item"

        # I'm going to assume the minimum and maximum damage are being passed correctly
        self.min_dmg: int = min_dmg
        self.max_dmg: int = max_dmg

Ok, that was a quick review of the `Adventurer` and `Item` classes. Now lets move on to the `Monster`. If you recall
from our diagram the `Monster` has a name, a `min_dmg` and `max_dmg` field, and a set of `hit_points`. I'm actually
going to create a class level function as well, called `create_monster`. Now, before I go on to described that, I want
you to just reflect on what I've said. I said I will create a class level function, and I didn't say I would create a
class method. So my question to you is, what's the difference between a class function and a class method?


Well, in both cases the underlying function being defined is associated with the class. But the method, or sometimes
call an instance method, takes an implicit first parameter of `self`. From this the method is able to interact with the
object it is being called on, and in the case of a `Monster` this means the method can do things like change the number
of `hit_points` which are available, or maybe the `name` of the monster. A class function does not have this first
parameter, so it is limited to manipulating the input parameters and returning some result. It still can create new
objects which are available in the interpreter, but it cannot change the `Monster` instance that it's associated with.
Class functions are often use to support specific design patterns, and I thought I would demonstrate one of these -- the
factory pattern -- with this object.


In [ ]:
# The class appears just as you might expect, and you see here I've included documentation using
# the google style docstrings. Note that since Python 3 is all unicode-based you don't have to limit
# yourself to just the letters A through Z, and you can see here I've decided to include a bit of
# flare through the use of emojis in my documentation.
class Monster:
    """A terrifying monster in a cave system!

    In this version of the game there are a handful of different monsters - Ghosts 👻, Goblins 👺,
    and the most dreaded monster of all... Pythons 🐍! To create a random monster which is one
    of these kinds call the function create_monster(), otherwise inherit from this class and
    build your own kind of monster!

    Attributes:
        name: The display name for the monster.
        min_dmg: The minimum damage the monster might cause, cannot be negative.
        max_dmg: The maximum damage the monster might cause, cannot be negative.
        hit_points: The total number of hit points a monster has, cannot be negative.
    """

    def __init__(self, name: str, min_dmg: int, max_dmg: int, hit_points: int) -> None:
        """Initializes the Monster with its attributes.

        Args:
            name: A description of the Item being used.
            min_dmg: The minimum damage the item can cause.
            max_dmg: The maximum damage the item can cause.
            hit_points: The total number of hit points a monster has.
        """
        # The __init__ method of the Monster is intentionally kept really simple. Instead of
        # embedding the logic for creating a monster here, I've put it in the class function
        # (not a method) below. Sometimes you will see this approach to object
        # creation as the "factory pattern", and that create_monster() is like a factory
        # that builds different kinds of monsters.

        # TODO: Things which haven't been done yet are often marked as TODO in code comments,
        # and this is a reminder that they need to be fixed! In this case, I have done no
        # input validation, such as ensuring there is a display name, or that that the min_dmg
        # is greater than zero but less than the max_damage. While using TODO is just a
        # convention, it's a great one and several tools such as github allow you to search or
        # create issues based on this string.

        # TODO: Add input validation for name, min/max dmg, and hit_points
        self.name: str = name
        self.min_dmg: int = min_dmg
        self.max_dmg: int = max_dmg
        self.hit_points: int = hit_points

    # This is a class function, not a method, and it is used to create a random monster.
    # We're using a "factory pattern" here, which is a common way to create objects in Python.
    def create_monster():
        """Creates a random monster!"""

        # A list of possible monsters, all with emojis to make them more fun!
        monsters = ["Ghost 👻", "Goblin 👺", "Python 🐍"]

        # Choose a monster from the list and then lets create it
        name = random.choice(monsters)
        if name == "Ghost 👻":
            # the weakest of the monsters, easy to beat!
            return Monster(name, random.randint(0, 1), random.randint(1, 2), 2)
        elif name == "Goblin 👺":
            # still pretty weak but has more hit points
            return Monster(name, random.randint(0, 1), random.randint(1, 2), 10)
        elif name == "Python 🐍":
            # the real foe of the dungeon, very strong, but low total hit points
            return Monster(name, random.randint(3, 6), random.randint(6, 12), 5)

Ok, normally I would show you how this class works, but instead of doing it through a video I want to jump right to
using this new `Monster` class with our `Rooms`. I've decided that when you create a new `Room` object it will
initialize itself with a monster and item through the `create_monster_and_treasure` function, and we already saw that on
our interaction diagrams. So we'll convert that to code here, and it's pretty straight forward. However, I want to add a
second method, called `enter`, which takes in an `Adventurer` object. This method is going to do the heavy lifting of
the game play, and it's a design decision I made that is worth talking about a bit.

You see, there's lots of different ways to model our problem space and write code to solve it, and they often require
unique trade offs that don't seem to change the functionality to our end users. In this case, I've decided that a `Room`
object knows almost everything it needs to know to determine whether combat was successful or not. It has the Monster,
it has the Item, and we can pass in the Adventurer. Put another way, I think that encapsulating the game play within the
`Room` class creates a readable and reasonable piece of code. But, it would be completely fine to put the combat
resolution in the `Adventurer` class as well. For instance, you could have a `fight` method of the `Adventurer` which
takes in a `Room` or even a `Monster`, resolves combat, and then awards the `Item` to the `Adventurer`. In this case I
feel like `Room` is a good place for the combat resolution -- maybe in the future we can have different kinds of `Room`
objects, all which have a different way of resolving combat.

Let's jump in and write this code.


In [ ]:
class Room:
    """A room in a cave system which an Adventurer can enter.

    The Room has a Monster and Item (treasure) when created. In addition, the Room has a method
    called enter, which allows the Adventurer to enter the room. Combat is immediately resolved
    when the Adventurer enters the room.

    Attributes:
        monster: The monster which is lurking in this room.
        treasure: An item which is in the room and give to the Adventurer when combat is resolved.
    """

    def __init__(self) -> None:
        """Initializes the Room with a random monster and item."""
        # The room initializes itself through a separate function called create_monster_and_treasure().
        # It's expected that future subclasses might override this method, and use their own
        # creation code, but I'll leave that up to you to do.
        self.create_monster_and_treasure()

    def create_monster_and_treasure(self) -> None:
        """Creates a random monster and item for the room."""
        # A list of possible items we might find. For items we are embedding the logic within
        # this function directly. Maybe I should turn these into emojis too, or maybe you would
        # like to? The choice is yours!
        items: [str] = ["Dagger", "Sword", "Club", "Heavy Block"]

        # Choose a random item as the treasure
        self.treasure = Item(
            name=random.choice(items),
            min_dmg=random.randint(0, 2),
            max_dmg=random.randint(4, 8),
        )

        # A monster in this room! For the monster we have embedded the logic in the Monster
        # class, putting it in the create_monster() function. Where you put the logic is up
        # to you -- in the case of the item we can imagine that we might want to create
        # news items that have nothing to do with rooms, so putting the logic inside of the
        # room function to restrict the items that we can find might make sense.

        # It's worth it to consider this for a moment -- there are many different possible ways
        # to organize your code, and I'm showing two different ways with the Item and Monster
        # here. A great exercise, should you want it, is to refactor (which means update) the
        # Item class so it has a create_item() class function as well, and then update the Room
        # to match.
        self.monster = Monster.create_monster()

    def enter(self, hero: Adventurer) -> str:
        """A method which pits a hero against the monster in the room.

        If the result of combat is that the hero has less than 0 hit points then the return value
        is "win", otherwise it is "lose".

        Args:
            hero: The Adventurer entering the room.
        """
        # Normally I wouldn't use print statements in a game like this as they are pretty inflexible,
        # but in this case I want the game to be playable by the end of the notebook, so lets
        # add some as log statements.
        print(f"{hero.name} enters the room and looks around...")

        # We need to be able to calculate the combat between the hero and the monster, and this
        # gives us another great opportunity to introduce some Python concepts: duck typing.
        # Duck typing is a way of programming where you don't care what the object is, you just
        # care that it has the right methods and attributes. In this case, I've defined a function
        # called calculate_damage that takes in two parameters, an attacker and a defender. I don't
        # need to know whether it's a monster attacking an adventurer, or an item (wielded by the
        # adventurer) attacking a monster. This is because I know that both the Item and the Monster
        # classes have a min_dmg and max_dmg attribute, and that the Monster and Adventurer classes
        # both have a hit_points attribute. This is a very powerful concept in Python, and you see
        # it used all over in the standard library and in many popular packages.  An alternative
        # to this approach is to use inheritance, and that would be common a common approach in other
        # programming languages such as Java (though Interfaces in Java would be reasonable as well).

        def calculate_damage(
            attacker: Item | Monster, defender: Adventurer | Monster
        ) -> bool:
            """Calculates the damage done by the attacker to the defender.

            Returns True if the combat has resulted in the death of the defender, False otherwise.

            Args:
                attacker: The attacker, which can be an Item or a Monster.
                defender: The defender, which can be an Adventurer or a Monster.
            """
            # calculate the damage done and hit points left
            damage_done: int = random.randint(attacker.min_dmg, attacker.max_dmg)
            defender.hit_points = defender.hit_points - damage_done

            # print out a script of what just happened
            print(
                f"{attacker.name} caused {damage_done} damage to {defender.name} which has {defender.hit_points} left!"
            )

            # if the defender has perished indicate combat is done
            if defender.hit_points < 0:
                return True
            return False

        # Ok, we're almost done the Room class! Now that we have a helper function to calculate
        # damage we can use it to resolve the combat between the hero and the monster. I'll create
        # a loop that just continues to pit hero and monster against one another until someone
        # perishes.
        combat_done = False
        while not combat_done:
            # Randomly pick the hero/monster to attack
            if random.randint(0, 1) == 1:  # hero goes first
                # Hero attacks monster with a random item in their bag. A fun extension to this
                # game would be to intelligently choose the best item in the bag, and/or to allow
                # items to break after a certain number of uses.
                item: Item = random.choice(hero.bag)
                combat_done = calculate_damage(item, self.monster)
            else:
                # monster attacks hero
                combat_done = calculate_damage(self.monster, hero)

        # If the adventurer is still alive, they won this room and get the item!
        if hero.hit_points >= 0:
            hero.bag.append(self.treasure)
            print(f"{hero.name} adds a {self.treasure.name} to their bag!")
            return "win"

        # Otherwise I'm afraid they perish
        return "lose"

Alright, we're almost there. We have Adventurers who have Items in their bag, Monsters which sit in Rooms gaurding
treasure, so now we just need to write the Cave class. This class is actually how we are going to start the game. If you
remember from our class diagram there is a list of Room objects in the Cave, so we need to create those in the Cave's
`__init__` method. We also need to add one more method called `explore`, which takes in the `Adventurer` object and runs
them through our cave system. This kind of function might be called a game loop in video game development, as it just
keeps iterating over steps of actions until the game has completed.


In [ ]:
class Cave:
    """A place for an Adventurer to Adventure!

    Caves have ten rooms, just like Adventurers have ten fingers.
    """

    def __init__(self):
        """Initializes the Cave with ten rooms."""
        self.rooms: [Room] = []
        for i in range(0, 10):
            self.rooms.append(Room())

    def explore(self, hero: Adventurer) -> None:
        """Runs an Adventurer through a Cave system.

        Args:
            hero: The Adventurer exploring the cave.
        """
        for room in self.rooms:
            result = room.enter(hero)
            if result == "lose":
                print(f"Sadly, {hero.name} has perished to the caves.")
                return
            else:
                print(
                    f"One more room explored, great job {hero.name}! You now have {len(hero.bag)} items in your bag and {hero.hit_points} hit points left.\n"
                )

        # if we made it this far the hero has survived!
        print(f"Congratulations {hero.name} in surviving the caves!")

In [ ]:
# Ok, that's it, now let's play this game!
hero = Adventurer("Sir Christopher")
cave = Cave()
cave.explore(hero)